In [12]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import KFold
import numpy as np
import time

# Load the MNIST dataset
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

# Combine training and test sets for 5-fold cross-validation
X = np.concatenate((X_train, X_test))
y = np.concatenate((y_train, y_test))

# Normalize pixel values to the range [0, 1]
X = X / 255.0

# Reshape data for the dense layers (flatten each 28x28 image to 784)
X = X.reshape(len(X), 28 * 28)


In [17]:
def cross_validate(loss_function, one_hot_encode=False):
    """
    Perform 5-fold cross-validation using the specified loss function.

    Args:
    - loss_function (str): Loss function to use for training.
    - one_hot_encode (bool): Whether to one-hot encode the labels (needed for KLD).

    Returns:
    - dict: Metrics including training and validation accuracy/loss and time taken for each fold.
    """
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    metrics = {"training_acc": [], "validation_acc": [], "training_loss": [], "validation_loss": [], "time": []}

    for train_idx, val_idx in kf.split(X):
        X_train_fold, X_val_fold = X[train_idx], X[val_idx]
        y_train_fold, y_val_fold = y[train_idx], y[val_idx]

        if one_hot_encode:
            y_train_fold = tf.keras.utils.to_categorical(y_train_fold, num_classes=10)
            y_val_fold = tf.keras.utils.to_categorical(y_val_fold, num_classes=10)

        # Define the model architecture as per Assignment 4 requirements
        model = keras.Sequential([
            keras.layers.Dense(16, input_shape=(784,), activation='sigmoid'),
            keras.layers.Dense(16, activation='sigmoid'),
            keras.layers.Dense(10, activation='softmax')
        ])

        # Compile the model with the specified loss function
        model.compile(optimizer='adam', loss=loss_function, metrics=['accuracy'])

        # Train the model and record the time taken
        start_time = time.time()
        model.fit(X_train_fold, y_train_fold, epochs=5, verbose=0)
        end_time = time.time()

        # Evaluate the model on training and validation data
        training_loss, training_acc = model.evaluate(X_train_fold, y_train_fold, verbose=0)
        validation_loss, validation_acc = model.evaluate(X_val_fold, y_val_fold, verbose=0)

        # Record metrics
        metrics["training_acc"].append(training_acc)
        metrics["validation_acc"].append(validation_acc)
        metrics["training_loss"].append(training_loss)
        metrics["validation_loss"].append(validation_loss)
        metrics["time"].append(end_time - start_time)

    return metrics


In [18]:
# Perform 5-fold cross-validation for sparse_categorical_crossentropy
sparse_metrics = cross_validate('sparse_categorical_crossentropy', one_hot_encode=False)

# Perform 5-fold cross-validation for categorical_crossentropy with one-hot encoding
kld_metrics = cross_validate('kld', one_hot_encode=True)


In [19]:
import numpy as np

def display_results(metrics, loss_function_name):
    """
    Display cross-validation results in a tabular format.

    Args:
    - metrics (dict): Cross-validation metrics.
    - loss_function_name (str): Name of the loss function for the results.
    """
    print(f"Results for {loss_function_name}:")
    print("{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}".format(
        "Fold", "Train Acc", "Val Acc", "Train Loss", "Val Loss", "Time Taken"
    ))

    # Print results for each fold
    for i in range(5):
        print("{:<15} {:<15.4f} {:<15.4f} {:<15.4f} {:<15.4f} {:<15.4f}".format(
            i + 1,
            metrics["training_acc"][i],
            metrics["validation_acc"][i],
            metrics["training_loss"][i],
            metrics["validation_loss"][i],
            metrics["time"][i]
        ))

    # Calculate and display average and standard deviation for accuracy and loss
    avg_train_acc = np.mean(metrics["training_acc"])
    avg_val_acc = np.mean(metrics["validation_acc"])
    avg_train_loss = np.mean(metrics["training_loss"])
    avg_val_loss = np.mean(metrics["validation_loss"])
    avg_time = np.mean(metrics["time"])

    std_train_acc = np.std(metrics["training_acc"])
    std_val_acc = np.std(metrics["validation_acc"])
    std_train_loss = np.std(metrics["training_loss"])
    std_val_loss = np.std(metrics["validation_loss"])
    std_time = np.std(metrics["time"])

    # Print averages and standard deviations
    print("\nAverage:")
    print("{:<15} {:<15.4f} {:<15.4f} {:<15.4f} {:<15.4f} {:<15.4f}".format(
        "Average", avg_train_acc, avg_val_acc, avg_train_loss, avg_val_loss, avg_time
    ))
    print("\nStandard Deviation:")
    print("{:<15} {:<15.4f} {:<15.4f} {:<15.4f} {:<15.4f} {:<15.4f}".format(
        "Std Dev", std_train_acc, std_val_acc, std_train_loss, std_val_loss, std_time
    ))
    print("\n")

# Example usage:
# Display results for sparse_categorical_crossentropy
display_results(sparse_metrics, "Sparse Categorical Crossentropy")

# Display results for kullback_leibler_divergence
display_results(kld_metrics, "Kullback-Leibler Divergence")


Results for Sparse Categorical Crossentropy:
Fold            Train Acc       Val Acc         Train Loss      Val Loss        Time Taken     
1               0.9376          0.9256          0.2199          0.2524          16.7379        
2               0.9425          0.9346          0.2023          0.2264          16.3957        
3               0.9406          0.9303          0.2096          0.2393          16.4421        
4               0.9409          0.9327          0.2103          0.2366          16.1845        
5               0.9389          0.9318          0.2163          0.2408          16.5117        

Average:
Average         0.9401          0.9310          0.2117          0.2391          16.4544        

Standard Deviation:
Std Dev         0.0017          0.0030          0.0061          0.0083          0.1789         


Results for Kullback-Leibler Divergence:
Fold            Train Acc       Val Acc         Train Loss      Val Loss        Time Taken     
1               0